In [2]:
using Pkg
Pkg.instantiate()

   Installed Libmount_jll ───────────────── v2.41.0+0
   Installed x265_jll ───────────────────── v3.5.0+0
   Installed GR_jll ─────────────────────── v0.73.16+0
   Installed libdecor_jll ───────────────── v0.2.2+0
   Installed JpegTurbo_jll ──────────────── v3.1.1+0
   Installed libfdk_aac_jll ─────────────── v2.0.3+0
   Installed LRUCache ───────────────────── v1.6.2
   Installed OffsetArrays ───────────────── v1.17.0
   Installed HypergeometricFunctions ────── v0.3.28
   Installed LERC_jll ───────────────────── v4.0.1+0
   Installed LoggingExtras ──────────────── v1.1.0
   Installed FFTW ───────────────────────── v1.9.0
   Installed Accessors ──────────────────── v0.1.42
   Installed ShowCases ──────────────────── v0.1.0
   Installed StatsFuns ──────────────────── v1.5.0
   Installed ContextVariablesX ──────────── v0.1.3
   Installed Opus_jll ───────────────────── v1.3.3+0
   Installed Xorg_xkbcomp_jll ───────────── v1.4.7+0
   Installed Unitful ────────────────────── v1.22.1
   Ins

In [3]:
using JLD2
X_train = load("../../dataset/imdb_dataset_prepared.jld2", "X_train")
y_train = load("../../dataset/imdb_dataset_prepared.jld2", "y_train")
X_test = load("../../dataset/imdb_dataset_prepared.jld2", "X_test")
y_test = load("../../dataset/imdb_dataset_prepared.jld2", "y_test")
embeddings = load("../../dataset/imdb_dataset_prepared.jld2", "embeddings")
vocab = load("../../dataset/imdb_dataset_prepared.jld2", "vocab")
nothing

embedding_dim = size(embeddings,1)

50

In [4]:
using Flux

model = Chain(
    Flux.Embedding(length(vocab), embedding_dim),
    x->permutedims(x, (2,1,3)),
    Conv((3,), embedding_dim => 8, relu),
    MaxPool((8,)),
    Flux.flatten,
    Dense(128, 1, σ)
)

Chain(
  Embedding(12849 => 50),               # 642_450 parameters
  var"#11#12"(),
  Conv((3,), 50 => 8, relu),            # 1_208 parameters
  MaxPool((8,)),
  Flux.flatten,
  Dense(128 => 1, σ),                   # 129 parameters
)                   # Total: 5 arrays, 643_787 parameters, 2.456 MiB.

In [5]:
#Wyciągnięcie wag
model[3].weight # Konwolucja i wagi wewnętrznego Dense?
# model[3].bias
# model[6].weight # Dense
# model[6].bias

3×50×8 Array{Float32, 3}:
[:, :, 1] =
  0.0476643  -0.033644    0.141272   …  -0.12473    0.08433      0.0543718
 -0.0927518   0.153988   -0.0920084      0.17204    0.00432104  -0.103841
 -0.0501074  -0.0429634   0.164857       0.121688  -0.00800216   0.00528064

[:, :, 2] =
 0.154323   0.0312806  0.110993   0.0953383  …   0.160177     0.0347331
 0.152952  -0.141098   0.0610598  0.0550829     -0.0198633   -0.149853
 0.161847  -0.0362219  0.135836   0.0910178     -0.00146301  -0.00598979

[:, :, 3] =
  0.0574825   0.0887202   0.0936802  …  0.106306    0.111454    -0.146857
 -0.0904946   0.17188     0.145748      0.135589   -0.0127035   -0.0664381
  0.128691   -0.10173    -0.136342      0.0825322  -0.00877334  -0.0975296

[:, :, 4] =
  0.148273   -0.146744     0.0568914  …  -0.0909555   0.123563   0.0640985
 -0.105571   -0.129751    -0.0145516      0.132692   -0.0254055  0.0783824
  0.0942351  -0.00440578  -0.0907788     -0.018809    0.131853   0.155455

[:, :, 5] =
  0.0283788  -0.13567

In [7]:
# add Glove embeddings to Embedding layer
model.layers[1].weight .= embeddings;

In [8]:
using Printf, Statistics

dataset = Flux.DataLoader((X_train, y_train), batchsize=64, shuffle=true)

loss(m, x, y) = Flux.Losses.binarycrossentropy(m(x), y)
accuracy(m, x, y) = mean((m(x) .> 0.5) .== (y .> 0.5))

opt = Optimisers.setup(Adam(), model)

epochs = 5
for epoch in 1:epochs
    total_loss = 0.0
    total_acc = 0.0
    num_samples = 0

    t = @elapsed begin
        for (x, y) in dataset
            grads = Flux.gradient(model) do m
                loss(m, x, y)
            end
            Optimisers.update!(opt, model, grads[1])
            total_loss += loss(model, x, y)
            total_acc += accuracy(model, x, y)
            num_samples += 1
        end

        train_loss = total_loss / num_samples
        train_acc = total_acc / num_samples

        test_acc = accuracy(model, X_test, y_test)
        test_loss = loss(model, X_test, y_test)
    end

    println(@sprintf("Epoch: %d (%.2fs) \tTrain: (l: %.2f, a: %.2f) \tTest: (l: %.2f, a: %.2f)", 
        epoch, t, train_loss, train_acc, test_loss, test_acc))
end

Epoch: 1 (40.13s) 	Train: (l: 0.54, a: 0.71) 	Test: (l: 0.41, a: 0.81)
Epoch: 2 (14.54s) 	Train: (l: 0.34, a: 0.85) 	Test: (l: 0.34, a: 0.86)
Epoch: 3 (14.46s) 	Train: (l: 0.26, a: 0.90) 	Test: (l: 0.32, a: 0.87)
Epoch: 4 (15.12s) 	Train: (l: 0.20, a: 0.93) 	Test: (l: 0.31, a: 0.87)
Epoch: 5 (14.39s) 	Train: (l: 0.15, a: 0.95) 	Test: (l: 0.33, a: 0.87)
